### Default Setting 

In [1]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.readers.chroma import ChromaReader
from llama_index import StorageContext, load_index_from_storage, load_indices_from_storage
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.llms import HuggingFaceLLM
from llama_index.node_parser import SentenceSplitter 
from llama_index.schema import MetadataMode
from IPython.display import Markdown, display
from llama_index import QueryBundle 
import chromadb
import pandas as pd 
import openai
import os
import getpass

In [2]:
data_path = os.path.join('/workspace/data/')
index_path = os.path.join('/workspace/db/local')

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key: ········


### Retrieve 

In [47]:
from llama_index.retrievers import VectorIndexRetriever 
from llama_index.vector_stores import SimpleVectorStore
from llama_index.query_engine import RetrieverQueryEngine 
from llama_index.postprocessor import SimilarityPostprocessor 
from llama_index.postprocessor import SentenceTransformerRerank 
from llama_index.postprocessor import KeywordNodePostprocessor 
from llama_index.postprocessor import SimilarityPostprocessor, CohereRerank
from llama_index.schema import Node, NodeWithScore 

In [48]:
desc_vectorstore = SimpleVectorStore.from_persist_path(os.path.join(index_path, 'desc', 'default__vector_store.json'))
features_vectorstore = SimpleVectorStore.from_persist_path(os.path.join(index_path, 'features', 'default__vector_store.json'))
qualification_vectorstore = SimpleVectorStore.from_persist_path(os.path.join(index_path, 'qualification', 'default__vector_store.json'))

In [52]:
desc_dict = desc_vectorstore.to_dict()
features_dict = features_vectorstore.to_dict()
qualification_dict = qualification_vectorstore.to_dict()

#### - Embedding Model  - kakaobank 

In [13]:
model_name = 'kakaobank/kf-deberta-base'
embed_model = HuggingFaceEmbedding(model_name=model_name)

In [14]:
desc_storage_context = StorageContext.from_defaults(persist_dir=os.path.join(index_path, 'desc'))
features_storage_context = StorageContext.from_defaults(persist_dir=os.path.join(index_path, 'features'))
qualification_storage_context = StorageContext.from_defaults(persist_dir=os.path.join(index_path, 'qualification'))

In [15]:
parser = SentenceSplitter(chunk_size=512, chunk_overlap=30)   # SentenceSplitter(chunk_size=1024, chunk_overlap=20)

In [16]:
embedding_service = ServiceContext.from_defaults(node_parser=parser, embed_model=embed_model, llm=None)

LLM is explicitly disabled. Using MockLLM.


In [17]:
# service_context 전달 안해주면 query 시 dimension 오류 발생 
features_idx = load_index_from_storage(features_storage_context, index_id='loan_tmp', service_context=embedding_service)
desc_idx = load_indices_from_storage(desc_storage_context, index_ids=['card_tmp', 'loan_tmp', 'deposit_tmp'], service_context=embedding_service)
qualification_idx = load_index_from_storage(qualification_storage_context, index_id='loan_tmp', service_context=embedding_service)

#### - Retriever 

In [18]:
retriever = VectorIndexRetriever(
    index = features_idx,
    service_context=embedding_service,
    similarity_top_k = 10, 
    verbose=True
)

In [19]:
def get_retrieved_nodes(
    retriever, query_str, vector_top_k=10, similarity_cutoff=0.6, reranker_top_n=3, service_context=None, with_reranker=False
):
    # query bundle 생성 
    query_bundle = QueryBundle(query_str)

    # 유사도가 제일 높은 node 추출 
    retrieved_nodes = retriever.retrieve(query_bundle)

    # 전처리  - 유사 점수 기준 Cutoff 
    node_postprocessors = SimilarityPostprocessor(similarity_cutoff=similarity_cutoff)
    processed_nodes = node_postprocessors.postprocess_nodes(retrieved_nodes)
    
    if with_reranker:   # 재순위화 
        reranker = SentenceTransformerRerank(
            model='bongsoo/albert-small-kor-cross-encoder-v1',
            top_n=reranker_top_n,
        )
        reranked_nodes = reranker.postprocess_nodes(
            processed_nodes, query_bundle
        )
    return reranked_nodes

In [43]:
with_reranker = True
cutoff = 0.4
query = '대출 금리가 가장 낮은 상품 정보 알려줘'

In [44]:
import time

start = time.time()
retrieved_nodes = retriever.retrieve(query)
nodes = get_retrieved_nodes(retriever, query, similarity_cutoff=cutoff, service_context=embedding_service, with_reranker=with_reranker)
end = time.time() - start 
print(end)

0.8045570850372314


In [50]:
def get_info(node):
    '''
    id, text, score 반환 
    '''
    id = features_dict['metadata_dict'][node.node_id]['document_id']
    txt = node.text 
    score = node.score 
    return [id, txt, score] 

In [45]:
print([node.text for node in retrieved_nodes])

['주택담보대출 이율변동이 걱정되시나요? 처음 금리가 대출만기까지 고정되는 순수장기고정금리 상품을 만나보세요!!', '주택담보대출 이율변동이 걱정되시나요? 처음 금리가 대출만기까지 고정되는 순수장기고정금리 상품을 만나보세요!!', '최장 50년간 고정금리 상품으로 금리변동에 따른 위험부담이 없으며, 아파트의 경우 소액임차보증금 공제없이 집값의 최대 70%까지 대출이 가능한 상품입니다.-기본형 : 대출실행일부터 만기까지 고정금리가 적용되는 상품-주택연금사전예약형 : 만40세 이상(본인 또는 배우자)고객이 보금자리론을 이용하면서 주택연금 가입을 사전예약하고 주택연금으로 전환 시 우대금리(연0.20%) 누적액을 장려금으로 지급하는 상품-입주자전용 : 신규 분양아파트 잔금대출 용도로 다음 요건을 충족하는 고객이 신청 가능*요건:1.(임시)사용승인 완료2.신청인이 최초 입주예정자(등기사항전부증명서상 최초 소유권자)3.당행 입주잔금대출 승인을 받은 사업장(신청 영업점에 문의)', '대출실행일부터 만기까지 안정적인 고정금리가 적용되는 상품으로 향후 금리 변동의 위험을 피하고자 하는 고객에게 적합한 상품개인', '변동금리 수준으로 금리우대 및 기준금리 변경 옵션을 부여한 대출상품', '당행 주택담보대출의 잔액만큼 대환가능한 상품으로서 고객의 대출금리를 낮춘 장기 고정금리 상품으로 향후 금리 변동의 위험을 피하고자 하는 고객에게 적합한 상품(인터넷 비대면 약정)', '담보대출을 사용하면 입출금통장에 대출이율과 같은 이율을 적용하여 최대 50%의 대출이자 절감효과(세전)를 누릴 수 있는 신개념 담보대출상품', '고정금리의 혜택과 변동금리의 혜택을 동시에 누릴수 있는 주택담보대출', '대출실행일부터 만기까지 안정적인 고정금리 주택담보대출.향후 금리 변동의 위험을 피하고자 하는 고객에게 적합한 상품.개인', '처음금리가 대출만기까지 고정되는 고정금리 주택자금대출이 부담되시나요? 5년마다 금리가 조정되는 New 순수장기고정금리 상품을 만나보세요!!']


In [46]:
contexts = [node.text for node in nodes]
scores = [node.score for node in nodes]
contexts, scores 

(['변동금리 수준으로 금리우대 및 기준금리 변경 옵션을 부여한 대출상품',
  '주택담보대출 이율변동이 걱정되시나요? 처음 금리가 대출만기까지 고정되는 순수장기고정금리 상품을 만나보세요!!',
  '주택담보대출 이율변동이 걱정되시나요? 처음 금리가 대출만기까지 고정되는 순수장기고정금리 상품을 만나보세요!!'],
 [0.23542526, 0.21586773, 0.21586773])

In [53]:
get_info(nodes[0])

['대출_510', '변동금리 수준으로 금리우대 및 기준금리 변경 옵션을 부여한 대출상품', 0.23542526]

### Augment 

In [54]:
chat_template = f"""
    사용자 쿼리와 유사한 문맥 정보는 다음과 같아. 
    ---------------------\n
    {contexts[0]}
    \n---------------------\n
    주어진 문맥 정보를 바탕으로, 사용자 질문에 대답해줘: {query}\n
    대답은 한국어로 하고, 주어진 문맥이 별로 도움되지 않는다면, 너가 대답을 만들어줘
"""

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
messages = [
{"role": "user", "content": "bbbbbb"},
{"role": "assistant", "content" : "aaaaa"},
{"role": "user", "content" : "bbbb"},
{"role": "assistant", "content" : "aaaaa"},
{"role": "user", "content" : "bbbbb"},
{"role": "assistant", "content" : "aaaaa"}
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False)
print(prompt)

### Generation 

In [55]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig
from llama_index.response_synthesizers import (
    ResponseMode,
    get_response_synthesizer,
)

In [56]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [59]:
llm = HuggingFaceLLM(
    context_window=512,
    max_new_tokens=128,
    generate_kwargs={"temperature": 1.0, "do_sample": True},
     tokenizer_name="davidkim205/komt-mistral-7b-v1",
    model_name="davidkim205/komt-mistral-7b-v1",
    device_map="cuda",
    # offload_folder='offload',
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 1024},
    # uncomment this if using CUDA to reduce memory usage
    # model_kwargs={"torch_dtype": torch.float16}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [60]:
mistral_service_context = ServiceContext.from_defaults(node_parser=parser, llm=llm)

responser = get_response_synthesizer(
    response_mode="compact",
    service_context=mistral_service_context,
    verbose=True,
    structured_answer_filtering=True,
)

In [61]:
contexts

['변동금리 수준으로 금리우대 및 기준금리 변경 옵션을 부여한 대출상품',
 '주택담보대출 이율변동이 걱정되시나요? 처음 금리가 대출만기까지 고정되는 순수장기고정금리 상품을 만나보세요!!',
 '주택담보대출 이율변동이 걱정되시나요? 처음 금리가 대출만기까지 고정되는 순수장기고정금리 상품을 만나보세요!!']

In [62]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=responser,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.3)],
)

In [64]:
query

'대출 금리가 가장 낮은 상품 정보 알려줘'

In [66]:
model_input = f"[INST] + {query} + [/INST]"

In [67]:
response = query_engine.query(model_input)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


> Refine context: 입출금통장에 대출이율과 같은 이율을 적용하여 최대 50%의 대출이자 절감효과(세전)를...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ValueError: Could not extract json string from output: 
Examples:
Given:
"You have found a list of loan companies online. Compare their loan products to see which ones offer the best rates. Each company has given you their respective interest rates for different loan terms. Which lender offers the lowest monthly payment for a $5000 loan term?"

In [58]:
model_name='davidkim205/komt-mistral-7b-v1'
config.base_model_name_or_path = model_name
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map="auto")

NameError: name 'config' is not defined

In [ ]:
generation_config = GenerationConfig(
    temperature=1,
    do_sample=True,
    top_p=0.95,
    max_new_tokens=512,
)

In [ ]:
gened = model.generate(
    **tokenizer(
        chat_template,
        return_tensors='pt',
        return_token_type_ids=False
    ).to('cuda'),
    generation_config=generation_config,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    streamer=streamer,
)

In [ ]:
result_str = tokenizer.decode(gened[0])

start_tag = f"[/INST]"
start_index = result_str.find(start_tag)

if start_index != -1:
    result_str = result_str[start_index + len(start_tag):].strip()

In [ ]:
print(result_str)

In [ ]:
user_input = '블랙-숄즈 모델(Black-Scholes Model)이 무엇인지 설명해주세요.'

In [ ]:
context_str = contexts[0]

In [ ]:
result = gen(contexts[0])

print('##########')
print(result)